In [3]:
# Import the necessary libraries
import pandas as pd
from pathlib import Path
from textblob import TextBlob
import re

%store -r

In [4]:
# Import the restored data
# %run PrepareData.ipynb

In [5]:
# Reset the index of the dataframe
tsla_tweets_df = tsla_tweets_df.reset_index()

In [7]:
# Clean the text
def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)#Remove @mentions
    text = re.sub(r'#', '', text)# Remove '#' hash tag
    text = re.sub(r'RT[\s]+', '', text)# Remove RT
    text = re.sub(r'http?:\/\/\S+', '', text)# Remove hyperlink
    text = re.sub(r'\$[A-Za-z0-9]+', '', text)# Remove $
    text = re.sub(r'https?:\/\/\S+', '', text)# Remove hyperlink
    
    return text

tsla_tweets_df['body'] = tsla_tweets_df['body'].apply(cleanTxt)

In [ ]:
# Creaet a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [10]:
# Apply the models to the cleaned data
tsla_tweets_df['Subjectivity'] = tsla_tweets_df['body'].apply(getSubjectivity)
tsla_tweets_df['Polarity'] = tsla_tweets_df['body'].apply(getPolarity)

In [11]:
# Check the sentiment analysis result
tsla_tweets_df[['Subjectivity','Polarity']]

,Subjectivity,Polarity
0,0.400000,-1.500000e-01
1,0.100000,0.000000e+00
2,0.477273,3.806818e-01
3,0.000000,0.000000e+00
4,0.300000,1.000000e+00
...,...,...
357537,0.666667,1.809524e-01
357538,0.733333,1.850372e-17
357539,0.850000,4.000000e-01
357540,0.333333,1.666667e-01


In [12]:
# Get the mean
tsla_sentiments_df_textblob = tsla_tweets_df[["Subjectivity","Polarity"]].copy()
tsla_sentiments_df_textblob['Date'] = pd.to_datetime(tsla_tweets_df['post_date'].dt.date, utc=False)
tsla_sentiments_df_textblob = tsla_sentiments_df_textblob.groupby('Date').mean()
display(tsla_sentiments_df_textblob.head())
display(tsla_sentiments_df_textblob.tail())

,Subjectivity,Polarity
Date,,
2015-01-01,0.400000,-0.150000
2015-01-02,0.406539,0.180581
2015-01-03,0.210845,0.046556
2015-01-04,0.201275,0.076732
2015-01-05,0.078016,0.031528


,Subjectivity,Polarity
Date,,
2019-12-28,0.386012,0.079870
2019-12-29,0.382562,0.072515
2019-12-30,0.392603,0.080258
2019-12-31,0.404880,0.081927
2020-01-01,0.415301,0.123758


In [ ]:
%store tsla_sentiments_df_textblob